#### Luis Ramirez
#### Problem Set 1
In this assingment I work on problems from a bioinformatics text by: Dr. Greg Caporaso

#### Chapter 2.1: Pairwise Sequence Alingment####
We explore sequence similarity

In [1]:
%pylab inline 
import numpy as np
from IPython.core.display import HTML
from IPython.core import page
page.page = print


Populating the interactive namespace from numpy and matplotlib


In [2]:
from scipy.spatial.distance import hamming
from skbio import DNA


r1 = DNA("ACCCAGGTTAACGGTGACCAGGTACCAGAAGGGTACCAGGTAGGACACACGGGGATTAA")
r2 = DNA("ACCGAGGTTAACGGTGACCAGGTACCAGAAGGGTACCAGGTAGGAGACACGGCGATTAA")
q1 = DNA("TTCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCA")

print(r1)
print(r2)
print(q1)

ACCCAGGTTAACGGTGACCAGGTACCAGAAGGGTACCAGGTAGGACACACGGGGATTAA
ACCGAGGTTAACGGTGACCAGGTACCAGAAGGGTACCAGGTAGGAGACACGGCGATTAA
TTCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCA


Storinng three DNA sequences as variables. Then we compute the hamming distance between them using the hamming() function.
    

In [ ]:
%psource hamming

print(hamming(r1,q1))
print(hamming(r2,q1))

0.2542372881355932
0.2711864406779661


we can observe that q1 has a smaller distance to r1 than it does to r2, so q1 is more similar to r1 than r2. However, it is not always this simple.

We assumed that only substitution events have occured, meaning one DNA base was substituted with another. So we define q2, which is similar to q1 except that a single base has been deleted at the beginning of the sequence. And a single base has been inserted at the end of the sequence. 

In [7]:
q2 = DNA("TCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCAT")

print(q1)
print(q2)

print(hamming(r1, q2))

TTCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCA
TCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCAT
0.7288135593220338


We note how the distance increased with two small changes. The deletion even at the beginning of `q2` shifted that sequence relative to `r1`, this may have resulted in the many bases "downstream" of the deleted bases being different. The large distance may not be biologically justified. 

What would be nice to do is to have a way to indicate that a deletion has occured in `q2`. Let's define `q3` where we use a `-` character to indicate a deletion with respect to `r1`. This might be a more reasonable result, lets see...

In [9]:
q3 = DNA("-TCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCA")

print(q1)
print(q3)

print(hamming(r1, q3))

TTCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCA
-TCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCA
0.2542372881355932


What we did created a pairwise alignment of `r1` and `q3`. In other words, we have aligned positions to maximize the similarity between the two sequences. We will refer `-` as gap characters. 

We can observe the alignment ourselves if we print the output. 

In [ ]:
print(r1)
print(q3)

#### 2.1.1: Answering what is a sequence alignemnt? 
Over the course of biological evolution, a DNA sequence changes. Frequently due to random errors in replication. These errors are known as mutations. Here are a few examples of mutations: Substitutions, where one base (or amino acid) is replaced. Insertions, where one or more contiguous bases are inserted into a sequence. Deletions, where one or more contiguous bases are deleted from a sequence.     

In [2]:
from show_F import show_F
%psource show_F

seq1 = ("")
seq2 = ("")

